# adjustments.ipynb

This is where data adjustments, editing, etc. occurs including:


*   adding id's to establishments
*   removal of uninterested data
*   fixing errors after collecting data (longitude/latitude flip)



# installs and imports

In [15]:
# installs and imports 

!pip install selenium  
!apt-get update
!apt-get upgrade
!apt install chromium-chromedriver
!apt-get update
!apt-get upgrade
!pip install import-ipynb
!apt-get update
!apt-get upgrade

# drive mounting, path appending
mnt = '/content/gdrive'
# ROOT = 'path/to/Austin_Drinks'
ROOT = 'gdrive/MyDrive/Austin_Drinks'
from google.colab import drive
drive.mount(mnt)
import sys
sys.path.append(ROOT)
sys.path.append(ROOT + '/Data')
sys.path.append(ROOT + '/Data/search_result_links')
sys.path.append(ROOT + '/Data/json_by_city')
sys.path.append(ROOT + '/Data/json_by_city_no_reviews_images')
sys.path.append(ROOT + '/Data/csv_by_city')
print(sys.path)

import import_ipynb
from datahandler import datacollector as dc

import time
import os
import re

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [16]:
SITES =   { 
            'yelp' :  {
                        # base url for yelp
                        'url':'https://www.yelp.com',

                        # in Method get_search_result_links            
                        # search_results_tags 
                        'search_results_tags' : 'h4',           
                        #'search_results_tags_class' : 'css-uvzfg9',

                        # get_search_results_ext_tag_and_attribute
                        'get_search_results_ext_tag_and_attribute' : {'class' : 'css-uvzfg9'},
                       
                        # next page tag in Method get_search_result_links
                        #'next_page_tag' : 'a', anchor tags contain the href attribute
                        'next_page_tag_attr_and_def' : {'class' : 'next-link navigation-button__09f24__m9qRz css-1pxws0l'},
                        'ad_characters' : '/adredir?',
                        #'next_page_tag_attr' : 'class',
                        #'next_page_tag_attr_def' : 'next-link navigation-button__09f24__m9qRz css-1pxws0l',
                        'remove_unwanted_characters_in_url' : '\?osq=+[A-Za-z]+',


                        # in get_yelp_establishment_information: 
                        'establishment_name_tag' : 'h1',
                        'establishment_name_tag_attr_and_def' : {'class' : 'css-1x9iesk'},                      
                       
                        # attributes of button to reveal accommodations and unaccommodations
                        'amenities_header_text' : 'Amenities and More',
                        'amenities_reveal_button_text' : 'Attribute',
                        'amenities_reveal_button_hider_attr' : 'aria-controls',
                        'amenities_reveal_button_tag_attr' : 'class',
                        'amenities_reveal_button_tag_attr_def' : ' css-gme0a6',
                       
                        # status text to determine if the establishment is permanently closed
                        'status_text' : 'this location has closed',
                       
                       
                        # tags containing coordinates information
                        'location_tag' : 'div', 
                        'location_tag_attr_and_def' : {'class' : ' container__09f24__fZQnf border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG'},
                        'location_string_start' : '.png%7C',
                        'location_string_end' : '&',
                        'location_string_delimiter' : '%2C',
                       

                        # tag information to get businesses url
                        'business_url_tag' : 'div',
                        'business_url_tag_attr_and_def' : {'class' : ' css-1vhakgw border--top__09f24__exYYb border-color--default__09f24__NPAKY'},
                        'business_url_href_tag' : 'a',
                        'business_url_href_tag_attr_and_def' : {'class' : 'css-1um3nx', 'target' : '_blank'},
                        #'business_url_tag_attr' : 'class',
                        #'business_url_tag_attr_def' : ' css-1vhakgw border--top__09f24__exYYb border-color--default__09f24__NPAKY',
                       
                        # tag information to get establishment's tags
                        'est_tags_tag' : 'span',
                        'est_tags_tag_attr_and_def' : {'class' : ' display--inline__09f24__c6N_k margin-r1__09f24__rN_ga border-color--default__09f24__NPAKY'},
                        #'est_tags_tag_attr' : 'class',
                        #'est_tags_tag_attr_def' : ' display--inline__09f24__c6N_k margin-r1__09f24__rN_ga border-color--default__09f24__NPAKY',
                       
                        # hours of operation table tag information
                        'hoO_tag' : 'table',
                        'hoO_tag_attr_and_def' : {'class' : ' hours-table__09f24__KR8wh table__09f24__J2OBP table--simple__09f24__vy16f'},
                        #'hoO_tag_attr' : 'class',
                        #'hoO_tag_attr_def' : ' hours-table__09f24__KR8wh table__09f24__J2OBP table--simple__09f24__vy16f',
                        'hoO_row_tag_attr_and_def' : {'class' : ' table-row__09f24__YAU9e'},
                        #'hoO_row_tag_attr' : 'class',
                        #'hoO_row_tag_attr_def' : ' table-row__09f24__YAU9e',
                       
                        # amenities and anti-amenities tag information
                        'amenities_tag' : 'span', 
                        #'amenities_tag_attr' : 'class', 
                        #'amenities_tag_attr_def' : ' css-1h7ysrc', 
                        'amenities_tag_attr_and_def' : {'class' : ' css-1p9ibgf'},
                        'anti_amenities_tag' : 'span', 
                        #'anti_amenities_tag_attr' : 'class', 
                        #'anti_amenities_tag_attr_def' : ' css-1ccncw',
                        'anti_amenities_tag_attr_and_def' : {'class' : ' css-qyp8bo'},
                        
                      }
           }

# pass the SITES dictionary to the package and setup constants for the package
dc.set_constants(SITES, ROOT) 

root dir: gdrive/MyDrive/Austin_Drinks
{'yelp': {'url': 'https://www.yelp.com', 'search_results_tags': 'h4', 'get_search_results_ext_tag_and_attribute': {'class': 'css-uvzfg9'}, 'next_page_tag_attr_and_def': {'class': 'next-link navigation-button__09f24__m9qRz css-1pxws0l'}, 'ad_characters': '/adredir?', 'remove_unwanted_characters_in_url': '\\?osq=+[A-Za-z]+', 'establishment_name_tag': 'h1', 'establishment_name_tag_attr_and_def': {'class': 'css-1x9iesk'}, 'amenities_header_text': 'Amenities and More', 'amenities_reveal_button_text': 'Attribute', 'amenities_reveal_button_hider_attr': 'aria-controls', 'amenities_reveal_button_tag_attr': 'class', 'amenities_reveal_button_tag_attr_def': ' css-gme0a6', 'status_text': 'this location has closed', 'location_tag': 'div', 'location_tag_attr_and_def': {'class': ' container__09f24__fZQnf border-color--default__09f24__NPAKY overflow--hidden__09f24___ayzG'}, 'location_string_start': '.png%7C', 'location_string_end': '&', 'location_string_delimiter'

# experiments and adjustments


# reformat dictionaries into a preferred order

In [17]:
# reformat dictionaries to include ids towards the top.

def reformat_json(data, with_id = False, with_images = True, with_reviews = True):
  '''
  # reformat json data to a preferred readable format
  '''
  new_data = {}
  new_data['name'] = data['name']
  if with_id:
    new_data['id'] = data['id']
  new_data['open_for_business'] = data['open_for_business']
  new_data['tags'] = data['tags']
  new_data['website'] = data['website']
  new_data['longitude'] = data['longitude']
  new_data['latitude'] = data['latitude']
  new_data['address'] = data['address']
  new_data['postal_code'] = data['postal_code']
  new_data['city'] = data['city']
  new_data['state'] = data['state']
  new_data['telephone'] = data['telephone']
  new_data['price'] = data['price']
  new_data['hours'] = data['hours']
  new_data['accommodations'] = data['accommodations']
  new_data['unaccommodations'] = data['unaccommodations']
  new_data['rating_value'] = data['rating_value']
  new_data['review_count'] = data['review_count']
  if with_images:
    new_data['image'] = data['image']
  if with_reviews:
    new_data['reviews'] = data['reviews']
  return new_data

# add a unique id to each establishment, NOTE datacollector should be updated to do this for all files after collecting data.

In [18]:
# add IDs to each establishment
# first add all names to a set
estNames = set()
allEstDetail = {}
estCounter = 0


city_files = dc.get_city_json_filenames()
print(city_files)
for file in city_files:
  city_est = dc.load_city_json(file)
  estDetail = {}

  for est in city_est:
    estNames.add(est['name'])
    if est['name'] not in allEstDetail:
      allEstDetail[est['name']] = {'cityCount': 1}
      estCounter += 1
    else:
      allEstDetail[est['name']]['cityCount'] += 1
  time.sleep(1)





['San Antonio.json', 'New Braunfels.json', 'Cedar Park.json', 'Georgetown.json', 'Fredericksburg.json', 'Round Rock.json', 'San Marcos.json', 'Wimberley.json', 'Austin.json', 'Dripping Springs.json', 'Marble Falls.json']


In [19]:
print(estCounter)

664


In [20]:
for key, val in allEstDetail.items():
  print(key + '\t\t\t' + str(val['cityCount']))

Ranger Creek Brewing and Distilling			1
Weathered Souls Brewing			1
Man Overboard Brewing			1
Künstler Brewing			1
Second Pitch Beer			1
The Dooryard			1
Freetail Brewing Co			1
Roadmap Brewing			1
Back Unturned Brewing			1
Hops and Hounds			1
Blue Star Brewing			2
Longtab Brewing			1
Alamo Beer Company			1
Busted Sandal Brewing Company			1
Craft Cruiser			1
Southerleigh Fine Food And Brewery			1
The Hoppy Monk			1
Black Laboratory Brewing			1
Freetail Brewing			2
Third Brewsday			1
Flying Saucer Draught Emporium			1
Guadalupe Brewing			2
Barrelman Brewing			1
Z BrewZ Beer			1
Hidden Tavern			1
Dorćol Distilling Co. Bar + Brewery			1
Islla Street Brewing			1
Big Hops Huebner			1
Cibolo Creek Brewing			1
Burleson Yard Beer Garden			1
Vista Brewing			5
Big Hops - Shaenfield			1
Yard House			5
Busted Sandal Brewing			1
The Modernist			1
New Braunfels Brewing Co			2
The Esquire Tavern			1
Bar 1919			1
Taste Of San Antonio			1
Cactus Land Brewing			1
The Friendly Spot			1
5 Stones Artisan B

In [21]:

# assign ids to each establishment
dictIDs = {}
id = 1
for name in estNames:
  dictIDs[name] = id
  print(name + '\t\t\t' + str(id))
  id += 1

Fredericksburg Brewing			1
Hamilton Pool Vineyards			2
Lustre Pearl East			3
Eddie V's Prime Seafood			4
Save The World Brewing			5
Inwood Estates Vineyards Inc			6
Gray Line Tours - San Antonio			7
Vine Vault Austin			8
Flat Creek Enoteca			9
Cured			10
Hye Rum			11
The Wheel			12
Salt Traders Coastal Cooking			13
SoCo Gingerbeer			14
The Brewtorium Brewery and Kitchen			15
Goodnight Loving Vodka Distillery + Tasting Room			16
Mort Subite			17
Odd Duck			18
Stout House			19
Wedding Oak Winery - Fredericksburg			20
Central District Brewing			21
Celis Brewery			22
The Growling			23
Hahne Estates Winery			24
Cultures Grill and Bar			25
Texas Winos			26
Cover 2			27
The Wine Cellar est. 2020			28
Farmhouse Vineyards Marketplace and Tipsy Trailer			29
Fall Creek Vineyards - Driftwood			30
Hill Country Wine and Music Festival			31
Thirsty Planet			32
Hidden Tavern			33
Esperanza Winery			34
Bike and Brew ATX			35
Moonshine Ridge Distillery			36
Smokey's Cigar Lounge			37
All About Central T

In [22]:

# iterate through each file
# add the unique ids to the establishments
for file in city_files:
  city_est = dc.load_city_json(file)
  new_city_est = []
  for est in city_est:
    est['id'] = dictIDs[est['name']]
    new_est = reformat_json(est, with_id = True)
    new_city_est.append(new_est)
  dc.save_city_json(file, new_city_est)
  time.sleep(1)

city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Antonio.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/New Braunfels.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Cedar Park.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Georgetown.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Fredericksburg.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Round Rock.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Marcos.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Wimberley.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Austin.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Dripping Springs.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Marble Falls.json


# removing reviews and images from collected data

In [27]:
# we do not want to overwrite the files with complete data collection
# when removing images and reviews
# so we will save it to another destination in google drive
# not using the datacollector.ipynb here

# NOTE: consider rewriting function in datacollector.ipynb to write to
# a different directory for saving edited data files

import io
import json
def save_city_json(filename, data):
  '''
  # save organized JSON data collected from site
  '''
  global ROOT
  if not filename.endswith('.json'):
    filename += '.json'
  with io.open(ROOT + '/Data/json_by_city_no_reviews_images/' + filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii = False, indent = 4)
    print('city json saved to: ' + ROOT + '/Data/json_by_city_no_reviews_images/' + filename)

In [28]:
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  new_city_est = []
  for est in city_est:
    new_est = reformat_json(est, with_id = True, with_images = False, with_reviews = False)
    new_city_est.append(new_est)
  #dc.save_city_json(file, city_est)  # this function writes to '/Data/json_by_city' by default
  save_city_json(file, new_city_est)
  time.sleep(3)

San Antonio.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/San Antonio.json
New Braunfels.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/New Braunfels.json
Cedar Park.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/Cedar Park.json
Georgetown.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/Georgetown.json
Fredericksburg.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/Fredericksburg.json
Round Rock.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/Round Rock.json
San Marcos.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/San Marcos.json
Wimberley.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city_no_reviews_images/Wimberley.json
Austin.json
city json saved to: gdrive/MyDrive/Aus

# finding NoneType datacollector is trying to iterate through in __get_yelp_est_info, this error rarely throws but exists for reasons unknown

In [ ]:
import json

def __get_yelp_est_info(site, link):
  '''
  # Method specifically tailored to yelp for collecting data from an establishment's page
  '''
  global SITES, PERMANENTLY_CLOSED, OPEN_FOR_BUSINESS
  siteJSON = None
  infoJSON = {}
  infoCSV = []  
  html = dc.getBS(link, site)
  if html is not None:

    # get script tags of interest
    # pretty standard for script tags of interest
    tags = html.find_all('script', {'type' : 'application/ld+json'}) 

    # which tag in the list of tags do we need?
    # whichever contains 'name' in the first set of keys for the json.
    
    jsonFound = False
    for tag in tags:
      siteJSON = json.loads(''.join(tag.contents))
      if 'name' in siteJSON and siteJSON['name'] != 'Yelp':
        jsonFound = True
        break      
      siteJSON = None

    if 'name' not in siteJSON:
      print('Error: key "name" not in siteJSON!')
      dc.__errorLog(link, 'Error: key "name" not in siteJSON!')

    elif jsonFound and 'name' in siteJSON: 
      # extract information in the JSON
      if siteJSON is not None:

        # name of establishment
        infoJSON['name'] = dc.__get_value_from_json(siteJSON, 'name')
        if not infoJSON['name']:
          infoJSON['name'] = dc.clean_string(html.find(SITES[site]['establishment_name_tag'], SITES[site]['establishment_name_tag_attr_and_def']).text)
        infoCSV.append(infoJSON['name'])
        print('Collecting information for ' + infoJSON['name'])
        
        # address
        infoJSON['address'] = dc.__get_value_from_json(siteJSON, 'address', 'streetAddress')
        if 'Get Directions' in infoJSON['address']:
          infoJSON['address'] = ''
        infoCSV.append(infoJSON['address'])

        # city
        infoJSON['city'] = dc.__get_value_from_json(siteJSON, 'address', 'addressLocality')
        infoCSV.append(infoJSON['city'])

        # state
        infoJSON['state'] = dc.__get_value_from_json(siteJSON, 'address', 'addressRegion')
        infoCSV.append(infoJSON['state'])

        # postal code
        infoJSON['postal_code'] = dc.__get_value_from_json(siteJSON, 'address', 'postalCode')
        infoCSV.append(infoJSON['postal_code'])
        
        # phone number
        infoJSON['telephone'] = dc.__get_value_from_json(siteJSON, 'telephone')
        infoCSV.append(infoJSON['telephone'])

        # dollar signs/price range
        infoJSON['price'] = dc.__get_value_from_json(siteJSON, 'priceRange')
        infoCSV.append(infoJSON['price'])

        # review rating
        # make string for app
        value = dc.__get_value_from_json(siteJSON, 'aggregateRating', 'ratingValue')
        infoJSON['rating_value'] = str(value)
        infoCSV.append(infoJSON['rating_value'])

        # number of reviews
        value = dc.__get_value_from_json(siteJSON, 'aggregateRating', 'reviewCount')
        infoJSON['review_count'] = str(value)
        infoCSV.append(infoJSON['review_count'])

        #infoJSON['image'] = dc.__get_value_from_json(siteJSON, 'image')

        # reviews on establishment
        infoJSON['reviews'] = dc.__get_value_from_json(siteJSON, 'review')


        # for some reason in Yelp's json data 
        # each reviewer's rating value is nested into another dictionary by itself
        # let's pull it out and have one less layer deep to go for it..
        for review in infoJSON['reviews']:
          value = review['reviewRating']['ratingValue']
          review['reviewRating'] = value
          review['description'] = dc.clean_review_string(review['description'])


        # menu or cuisine, if available?
        infoJSON['cuisine'] = dc.__get_value_from_json(siteJSON, 'servesCuisine')

        # end json data extraction

  #    else:
        # extract information from the page itself
  ### TODO: EXTRACT INFORMATION FROM THE PAGE ITSELF IF JSON CANNOT BE FOUND
      # extracted data not in available json

      # check if the establishment is permanently closed
      if SITES[site]['status_text'] in html.text.lower():
        infoJSON['open_for_business'] = 0
      else:
        infoJSON['open_for_business'] = 1
      infoCSV.append(infoJSON['open_for_business'])

      # map location
      tag = html.find(SITES[site]['location_tag'], SITES[site]['location_tag_attr_and_def'])
      longitude = 0.0
      latitude = 0.0
      if tag is not None and tag.find('img')['src']:
        string = tag.find('img')['src']
        # %2C is the divider between longitude and latitude
        # there are multiple coordinates in this string
        # and the correct coordinates follow .png towards the end of the string.
        # layout: .png%7C00.000000%2C-00.000000&
        # .png%7C[]%2C[]&
        regex = re.search(SITES['yelp']['location_string_start'] + '+[A-Z0-9%.-]+' + SITES['yelp']['location_string_end'], string)

        if regex is not None:
          regex_str = regex.group(0)
          regex_str = re.sub(SITES['yelp']['location_string_start'], '', regex_str)
          regex_str = re.sub(SITES['yelp']['location_string_end'], '', regex_str)
          long_lat = regex_str.split(SITES['yelp']['location_string_delimiter'])
          longitude = long_lat[1] # longitude is second in regex_str
          latitude = long_lat[0] # latitude is first in regex_str
      infoJSON['longitude'] = float(longitude)
      infoJSON['latitude'] = float(latitude)
      infoCSV.append(longitude)
      infoCSV.append(latitude)

      # website
      tag = html.find(SITES[site]['business_url_tag'], SITES[site]['business_url_tag_attr_and_def'])
      website = ''
      if tag is not None and tag.find('a'):
        tagText = tag.find('a').get_text()
        if tagText is not 'Get Directions' and tagText:
          # some url strings are too long to publish, indicated be an ellipsis
          # if that is not the case, the tagText should be a valid url
          if int(bool(re.search(r'(\w+)\.{3,}', tagText))) == 1:
            website = tagText
          # otherwise, we get the href to the redirect link and parse
          else:
            tag = html.find(SITES[site]['business_url_href_tag'], SITES[site]['business_url_href_tag_attr_and_def'])
            if tag is not None:
              website = tag['href']        
              regres = re.search('url=(.*)&cache', tag['href'])
              result = regres.group(1)
              result = re.sub('%2F', '/', result)
              result = re.sub('%3A', ':', result)
              result = result.split('%', 1)[0]
              website = result
      infoJSON['website'] = website
      infoCSV.append(website)

      # est. type
      tags = html.find_all(SITES[site]['est_tags_tag'], SITES[site]['est_tags_tag_attr_and_def'])
      biz_tags = []
      tag_str = ''
      for tag in tags:
        anc_tags = tag.find_all('a')
        if anc_tags:
          for anc in anc_tags:
            if anc.get_text() != 'Unclaimed':
              tag_str += anc.get_text() + '|'    # cannot use commas as a delimiter because we are currently saving to csv format.
              biz_tags.append(anc.get_text())

      infoJSON['tags'] = biz_tags      
      infoCSV.append(tag_str)

      # hours of operations, if available  
        # this is specifically for yelp
        # <table class = ' hours-table__373c0__1S9Q_ table__373c0__1paZL table--simple__373c0__3hEOO' ...>
        # yes, there is a space at the front
          # <tr class = ' table-row__373c0__3xT3x' ...>
            # <th class = ' table-header-cell__373c0__OywTx' ...> gets the day of the week
            # <td class = ' table-cell__373c0__Hc_7A table-cell--top__373c0__1hgmO' ...> gets the hours of the day
            # and td of same class above also gets for current day "Open", "Closed now", etc.. if text is present, blank otherwise.

      days_hours = {}
      day = ''
      hours = ''
      table = html.find(SITES[site]['hoO_tag'], SITES[site]['hoO_tag_attr_and_def'])
      if table is not None and table.find('tbody').contents:
        table_row = table.find_all('tr', SITES[site]['hoO_row_tag_attr_and_def'])
    #    arr = []
        for row in table_row:
          day_str = ''
          hours_str = ''
          # get th and td
          if row.find('th'):
            day_str = row.find('th').get_text()
            day += day_str + '|'
          else:
            day += '|'
          if row.find('td'):
            hours_str = row.find('td').get_text()
            hours_str = re.sub('(Next day)', '', hours_str)
            hours += hours_str + '|'
          else:
            hours += '|'        
          days_hours[day_str] = hours_str
    #      a = []
    #      a.append(day)   # 1 column
    #      a.append(hours)   # 1 column, only getting the first of two.
    #      arr.append(a)  
    #    data = pd.DataFrame(arr, columns = ['Day', 'Hours'])
    #    print(data)
      else:
  #        print('No hours table is available for ' + name) 
        # empty columns for 'day' and 'hours'.
        day = '||||||'
        hours = '||||||'

      # even if days_hours is empty, append it to the json file.
      infoJSON['hours'] = days_hours
      infoCSV.append(day)
      infoCSV.append(hours)

      # Amenities, if it exists.
      accommodation_list = []
      unaccommodation_list = []
      accomm_str = ''
      unaccom_str = ''
      if SITES[site]['amenities_header_text'] in html.text:
        # what the company accommodates
        amenities_tags = html.find_all(SITES[site]['amenities_tag'], SITES[site]['amenities_tag_attr_and_def'])
        # what the company does not accommodate
        anti_amenities_tags = html.find_all(SITES[site]['anti_amenities_tag'], SITES[site]['anti_amenities_tag_attr_and_def'])
        accommodation_list = [tag.text for tag in amenities_tags]
        unaccommodation_list = [tag.text for tag in anti_amenities_tags]
        for tag in amenities_tags:
          accomm_str += tag.text + '|'
        for tag in anti_amenities_tags:
          unaccom_str += tag.text + '|'
      infoJSON['accommodations'] = accommodation_list
      infoJSON['unaccommodations'] = unaccommodation_list
      infoCSV.append(accomm_str)
      infoCSV.append(unaccom_str)
        # about the business?

    else:
      print('ERROR: No json found on this URL!')
      dc.__errorLog(link, '')
  else:
    print('ERROR: getting BeautifulSoup object for ' + link)
    dc.__errorLog(link, 'Error getting BeautifulSoup object in __get_yelp_est_info')

  # reformat the json structure for readability before returning the object
  return infoCSV, dc.reformat_json(infoJSON)

In [ ]:
dataCSV, dataJSON = __get_yelp_est_info('yelp', 'https://www.yelp.com/biz/revolution-spirits-austin')

Building BS object at url @ https://www.yelp.com/biz/revolution-spirits-austin


In [ ]:
print(dataJSON)

{'name': 'Revolution Spirits', 'open_for_business': 1, 'tags': ['Distilleries'], 'website': 'http://revolutionspirits.com/', 'longitude': -98.010599, 'latitude': 30.235888, 'address': "12345 Paul's Valley RdBldg G and H", 'postal_code': '78737', 'city': 'Austin', 'state': 'TX', 'telephone': '(512) 358-1203', 'price': 'Moderate', 'hours': {'Mon': 'Closed', 'Tue': 'Closed', 'Wed': 'Closed', 'Thu': 'Closed', 'Fri': '1:00 PM - 7:00 PM', 'Sat': '1:00 PM - 7:00 PM', 'Sun': '1:00 PM - 7:00 PM'}, 'accommodations': ['Offers Takeout', 'Masks required', 'Staff wears masks', 'Many Vegetarian Options', 'Accepts Credit Cards', 'Accepts Apple Pay', 'Offers Catering', 'Private Lot Parking', 'Bike Parking', 'Wheelchair Accessible', 'Gender-neutral restrooms', 'Offers Military Discount', 'Open to All'], 'unaccommodations': ['No Delivery', 'Accepts Android Pay', 'Accepts Cryptocurrency', 'No Happy Hour'], 'rating_value': '5', 'review_count': '39', 'reviews': [{'author': 'Everett T.', 'datePublished': '20

In [ ]:
dataCSV, dataJSON = __get_yelp_est_info('yelp', 'https://www.yelp.com/biz/oddwood-ales-austin')

Building BS object at url @ https://www.yelp.com/biz/oddwood-ales-austin


In [ ]:
print(dataJSON)

{'name': 'Oddwood Ales', 'open_for_business': 1, 'tags': ['Breweries', 'Pizza'], 'website': 'http://oddwoodales.com', 'longitude': -97.70531, 'latitude': 30.286918, 'address': '3108 Manor Rd', 'postal_code': '78723', 'city': 'Austin', 'state': 'TX', 'telephone': '', 'price': '$11-30', 'hours': {'Mon': 'Closed', 'Tue': '3:00 PM - 11:00 PM', 'Wed': '3:00 PM - 11:00 PM', 'Thu': '3:00 PM - 11:00 PM', 'Fri': '12:00 PM - 11:00 PM', 'Sat': '12:00 PM - 11:00 PM', 'Sun': '12:00 PM - 9:00 PM'}, 'accommodations': ['Health Score', 'Offers Takeout', 'Masks required', 'Many Vegetarian Options', 'Accepts Credit Cards', 'Accepts Apple Pay', 'Outdoor Seating', 'Moderate Noise', 'Good For Kids', 'Street Parking, Private Lot Parking', 'Bike Parking', 'Dogs Allowed', 'Wheelchair Accessible', 'TV', 'Open to All'], 'unaccommodations': ['Accepts Android Pay', 'Accepts Cryptocurrency', 'Offers Catering'], 'rating_value': '4.5', 'review_count': '104', 'reviews': [{'author': 'Puja C.', 'datePublished': '2021-11

# locating the tag with the preferred json data.

In [ ]:
site = 'https://www.yelp.com/biz/central-district-brewing-austin'
html = dc.getBS(site, 'yelp')

Building BS object at url @ https://www.yelp.com/biz/central-district-brewing-austin


In [ ]:
import json
tags = html.find_all('script', {'type' : 'application/ld+json'}) 
for tag in tags:
  siteJSON = json.loads(''.join(tag.contents))
  if 'name' in siteJSON and siteJSON['name'] != 'Yelp':
    print(tag.prettify())  
    print(siteJSON['name'])
  print('\n\n')







<script type="application/ld+json">
 {"@context":"https://schema.org","@type":"LocalBusiness","name":"Central District Brewing","image":"https://s3-media0.fl.yelpcdn.com/bphoto/kd10SXJvizPD00NsbqCqSg/l.jpg","priceRange":"Moderate","telephone":"(512) 993-4511","address":{"streetAddress":"417 Red River St","addressLocality":"Austin","addressCountry":"US","addressRegion":"TX","postalCode":"78701"},"review":[{"author":"Scott N.","datePublished":"2022-03-12","reviewRating":{"ratingValue":5},"description":"Central District is a great place to grab a quick beer for happy hour or get some work done. They have a generous selection of local, craft beers, including some that they brew in house. If you like sours, you can get a flight and try all of them! \n\nIn addition to plenty of space for large groups or meetings, there are plenty of outlets around for getting some work done. The staff were friendly and extremely fast. Parking can be an issue in this area, but there are plenty of paid l

# getting redirect links and regexing the urls from them

In [ ]:
# used to get website_redirects and find a regex solution
# once one is found, we will replace website with solution

file = 'Marble Falls.json'
print(file)
city_est = dc.load_city_json(file)
for est in city_est:
  if 'website_redir' in est:
    print(est['website_redir'])
  else:
    print(est['name'] + ' does not have a website redirect!')

Marble Falls.json
Fiesta Winery does not have a website redirect!
Perissos Vineyard and Winery does not have a website redirect!
Spicewood Vineyards does not have a website redirect!
Flat Creek Enoteca does not have a website redirect!
Iron Wolf Ranch and Distillery does not have a website redirect!
Heart of Texas Wine Tours does not have a website redirect!
Horseshoe Bay Resort does not have a website redirect!
AJE Transportation does not have a website redirect!
Round Mountain Distilling does not have a website redirect!
Texas Tipsy Tours does not have a website redirect!
Taste of the Hills does not have a website redirect!
Texan Wine and Spirit Tours does not have a website redirect!
ThunderDome Party Bus does not have a website redirect!
Discover Texas Wine Tours does not have a website redirect!
Austin Nites Party Bus does not have a website redirect!
Cool Bus Atx does not have a website redirect!
All About Central Texas does not have a website redirect!
Save The World Brewing doe

In [ ]:
# first step, we want all of the expression 
# after 'url=' and before the first '&' character
# then we can parse out 'http' 'https' and '%1A%2B%3C'
# that should be enough to get a url for the app to use.
file = 'Marble Falls.json'
print(file)
city_est = dc.load_city_json(file)
for est in city_est:
  if 'website_redir' in est and est['website_redir'] :
    print('\n' + est['website_redir'])
    regres = re.search('url=(.*)&cache', est['website_redir'])
    print(regres)
    result = regres.group(1)
    result = re.sub('%2F', '/', result)
    result = re.sub('%3A', ':', result)
    result = result.split('%', 1)[0]
    print(result)
    print('website_redir has been regexed')
    print(est['website'])
    if int(bool(re.search(r'(\w+)\.{3,}', est['website']))) or est['website'] == 'Get Directions' or not est['website']:
      print('replacing website')
      est['website'] = result
  else:
    print(est['name'] + ' does not have a website redirect!\n')
  
dc.save_city_json(file, city_est)

Marble Falls.json
Fiesta Winery does not have a website redirect!

Perissos Vineyard and Winery does not have a website redirect!

Spicewood Vineyards does not have a website redirect!

Flat Creek Enoteca does not have a website redirect!

Iron Wolf Ranch and Distillery does not have a website redirect!

Heart of Texas Wine Tours does not have a website redirect!

Horseshoe Bay Resort does not have a website redirect!

AJE Transportation does not have a website redirect!

Round Mountain Distilling does not have a website redirect!

Texas Tipsy Tours does not have a website redirect!

Taste of the Hills does not have a website redirect!

Texan Wine and Spirit Tours does not have a website redirect!

ThunderDome Party Bus does not have a website redirect!

Discover Texas Wine Tours does not have a website redirect!

Austin Nites Party Bus does not have a website redirect!

Cool Bus Atx does not have a website redirect!

All About Central Texas does not have a website redirect!

Save The 

# flip longitude and latitude, fix an error in understanding which was which.  the datacollector package has been updated to correctly store these values.

In [ ]:
# used to flip longitude and latitude
'''
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    tmp = est['longitude']
    est['longitude'] = est['latitude']
    est['latitude'] = tmp
  dc.save_city_json(file, city_est)
  time.sleep(3)
'''

"\ncity_files = dc.get_city_json_filenames()\nfor file in city_files:\n  print(file)\n  city_est = dc.load_city_json(file)\n  for est in city_est:\n    tmp = est['longitude']\n    est['longitude'] = est['latitude']\n    est['latitude'] = tmp\n  dc.save_city_json(file, city_est)\n  time.sleep(3)\n"

# convert integer and float values to strings

In [ ]:
# used to convert integers and decimals to string
# scratch that, this has been resolved on swift end.
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    tmp = est['rating_value']
    est['rating_value'] = str(tmp)
    tmp = est['review_count']
    est['review_count'] = str(tmp)
  dc.save_city_json(file, city_est)
  time.sleep(3)

Marble Falls.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Marble Falls.json
San Antonio.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Antonio.json
New Braunfels.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/New Braunfels.json
Cedar Park.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Cedar Park.json
Georgetown.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Georgetown.json
Fredericksburg.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Fredericksburg.json
Round Rock.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Round Rock.json
San Marcos.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/San Marcos.json
Wimberley.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Wimberley.json
Austin.json
city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Austin.json


In [ ]:
# used to fix reviewRating values
file = 'Marble Falls.json'
city_est = dc.load_city_json(file)
for est in city_est:
  for review in est['reviews']:
    tmp = review['reviewRating']
    review['reviewRating'] = str(tmp)
dc.save_city_json(file, city_est)

city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Marble Falls.json


In [ ]:
# make review counts in Marble Falls strings.
file = 'Marble Falls.json'
city_est = dc.load_city_json(file)
for est in city_est:
  tmp = est['review_count']
  est['review_count'] = str(tmp)
dc.save_city_json(file, city_est)

city json saved to: gdrive/MyDrive/Austin_Drinks/Data/json_by_city/Marble Falls.json


# changed file names for data collected to make the app easier to work with

In [ ]:
# change file names for app.
'''
import os
import re

files = os.listdir(ROOT + '/Data/json_by_city')
for file in files:
  newname = re.sub('_', '', file)
  newname = re.sub('TX', '', file)
  os.rename(ROOT + '/Data/json_by_city/' + file, ROOT + '/Data/json_by_city/' + newname)
'''

"\nimport os\nimport re\n\nfiles = os.listdir(ROOT + '/Data/json_by_city')\nfor file in files:\n  newname = re.sub('_', '', file)\n  newname = re.sub('TX', '', file)\n  os.rename(ROOT + '/Data/json_by_city/' + file, ROOT + '/Data/json_by_city/' + newname)\n"

# explore the data collected

In [ ]:
# see all establishment tags
city = 'austin_TX.json'
city_files = dc.get_city_json_filenames()
tags = set()
city_est = dc.load_city_json(city)
print(city)
for est in city_est:
  for tag in est['tags']:
    tags.add(tag)
print(tags)

FileNotFoundError: ignored

In [ ]:
# see all days of the week strings

days = set()
city_files = dc.get_city_json_filenames()
for file in city_files:
  print(file)
  city_est = dc.load_city_json(file)
  for est in city_est:
    for days_dict in est['hours'].keys():
      days.add(days_dict)
  dc.save_city_json(file, city_est)
  time.sleep(3)